In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = np.load('/content/drive/MyDrive/PurpleAirSF-1H/data.npy')

In [ ]:
data = np.load('/content/drive/MyDrive/PurpleAirSF-10M/data.npy')

In [ ]:
data = np.load('/content/drive/MyDrive/PurpleAirSF-6H/data.npy')

In [ ]:
import math
sum = 0

sensors = len(data)
time = len(data[0])
f = 0
#format as [time index][features]
processData = np.zeros((time, sensors*4))
for l in range(4):
  f = l
  if l == 3:
    f = 11
  for j in range(sensors):
    i = sensors*l + j
    for k in range(time):
      if(not math.isnan(data[j][k][f])):
          processData[k][i] = data[j][k][f]
      else:
        sum = sum + 1
        processData[k][i] = data[j - 1][k][f]
        if(math.isnan(processData[k][i] and i < sensors)):
          processData[k][i] = data[j + 1][k][f]
        if(math.isnan(processData[k][i])):
          processData[k][i] = 0






In [ ]:
import pandas as pd

df = pd.DataFrame(processData)

In [ ]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,918,919,920,921,922,923,924,925,926,927
2276,58.400,36.100,28.000,31.066,81.900,60.467,60.433,29.534,65.767,38.350,...,0.6660,1.3660,1.3025,2.390,0.408,2.1050,0.328,0.5260,0.686,2.1545
2277,64.200,41.000,27.000,34.000,96.367,68.934,63.300,31.000,78.200,40.106,...,0.3620,0.4605,0.4010,4.590,0.247,1.1545,0.157,0.3190,0.156,0.9630
2278,49.867,41.166,27.767,35.000,79.367,64.534,47.200,34.966,63.633,40.989,...,0.2600,0.8610,0.5555,0.623,0.073,0.5210,0.042,0.1545,0.219,0.4540
2279,54.500,38.400,30.534,29.500,63.633,55.767,60.100,33.166,53.633,39.522,...,0.3530,1.1310,1.0985,2.220,0.618,1.0815,1.030,0.2340,0.966,1.2250
2280,63.300,37.466,29.000,31.000,83.200,60.566,66.867,32.000,64.567,38.578,...,0.7645,2.7145,1.5320,7.281,0.584,2.5915,1.387,0.7195,1.055,2.6840


In [ ]:
target_sensor = 108 + sensors*3
features = list(df.columns.difference([target_sensor]))

forecast_lead = 6
target = f"{target_sensor}_lead{forecast_lead}"

df[target] = df[target_sensor].shift(-forecast_lead)
df = df.iloc[:-forecast_lead]

df.head()

,0,1,2,3,4,5,6,7,8,9,...,919,920,921,922,923,924,925,926,927,804_lead6
0,60.433,43.733,37.633,28.066,77.200,64.133,60.900,27.933,68.966,45.544,...,1.9690,1.1665,1.828,3.206,1.8840,1.458,1.5695,1.846,1.0010,0.313
1,38.133,45.466,38.000,32.966,78.767,63.067,39.400,32.000,72.067,46.994,...,1.3330,0.5350,1.677,0.618,2.0960,0.785,2.4085,0.271,1.1775,0.442
2,23.900,43.834,37.534,33.000,50.666,43.600,19.733,33.000,57.334,46.928,...,1.4875,0.5330,1.277,0.208,1.3285,0.542,1.0725,0.599,1.4315,0.357
3,42.000,37.700,37.000,23.867,34.367,39.166,42.333,27.166,35.433,45.544,...,1.0940,0.8785,1.406,0.382,0.9325,1.481,0.8870,2.693,0.9445,0.414
4,65.233,42.000,38.800,25.233,75.833,62.267,69.900,28.066,68.834,45.667,...,2.2195,1.9815,2.493,1.375,2.4870,2.576,2.2945,3.747,1.5575,0.283


In [ ]:
test_start = time*9/10

df_train = df.loc[:test_start].copy()
df_test = df.loc[test_start:].copy()

print("Test set fraction:", len(df_test) / len(df))

df_train.tail()


Test set fraction: 0.09758241758241758


,0,1,2,3,4,5,6,7,8,9,...,919,920,921,922,923,924,925,926,927,804_lead6
2048,78.000,32.566,21.000,23.000,79.967,54.700,79.200,23.000,60.633,38.061,...,0.7985,0.2205,1.498,0.053,0.3915,0.171,0.1400,0.442,0.3935,0.071
2049,76.100,35.566,20.000,23.500,74.966,54.067,74.700,25.133,58.466,38.439,...,0.2670,0.0685,0.254,0.037,0.2420,0.015,0.0375,0.029,0.1865,0.071
2050,72.167,36.067,20.000,25.000,61.933,50.100,72.166,23.966,54.767,39.578,...,0.1150,0.0615,0.105,0.025,0.1400,0.029,0.0840,0.343,0.2050,0.200
2051,71.633,31.067,20.666,24.133,50.534,46.967,69.334,24.166,46.233,39.044,...,0.2295,0.1840,3.124,0.048,0.1590,0.099,0.0575,0.057,0.2150,0.199
2052,73.067,30.500,21.000,24.867,69.167,54.133,71.066,22.000,56.100,39.578,...,0.5380,0.4010,0.243,0.041,0.3190,0.271,0.1165,0.299,0.2400,0.185


In [ ]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()
print(target_stdev)
sum = 0

for c in df_train.columns:
    mean = df_train[c].mean()
    stdev = df_train[c].std()

    df_train[c] = (df_train[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev
    sum = sum + 1


df_train.tail()

0.3733583189897946


,0,1,2,3,4,5,6,7,8,9,...,919,920,921,922,923,924,925,926,927,804_lead6
2048,1.862431,-1.708292,-1.943784,-1.409114,0.813770,0.034858,1.780834,-1.284445,0.091194,-1.337869,...,-0.481109,-0.922621,0.227265,-0.828140,-0.851388,-0.582456,-0.744076,-0.536617,-0.684024,-0.737820
2049,1.731537,-1.098035,-2.143939,-1.311549,0.530847,-0.020518,1.488951,-0.943778,-0.077952,-1.239890,...,-0.798992,-1.169242,-0.563104,-0.873079,-0.965204,-0.838912,-0.909992,-1.072813,-0.786845,-0.737820
2050,1.460585,-0.996122,-2.143939,-1.018854,-0.206471,-0.367553,1.324588,-1.130162,-0.366681,-0.944657,...,-0.889901,-1.180599,-0.657770,-0.906783,-1.042858,-0.815897,-0.834723,-0.665148,-0.777655,-0.392307
2051,1.423796,-2.013217,-2.010636,-1.188032,-0.851349,-0.641630,1.140897,-1.098220,-1.032808,-1.083071,...,-0.821420,-0.981843,1.260337,-0.842184,-1.028393,-0.700820,-0.877618,-1.036461,-0.772688,-0.394986
2052,1.522587,-2.128556,-1.943784,-1.044806,0.202779,-0.014744,1.253239,-1.444157,-0.262632,-0.944657,...,-0.636910,-0.629760,-0.570093,-0.861844,-0.906583,-0.418061,-0.782116,-0.722273,-0.760270,-0.432483


In [ ]:
import torch
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=24):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[target].values).float()
        self.X = torch.tensor(dataframe[features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i):
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
torch.manual_seed(101)

batch_size = 4
sequence_length = 12

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([4, 12, 447])
Target shape: torch.Size([4])


In [ ]:
from torch import nn

class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 2

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,dropout = .5 ,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()

        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

In [ ]:
learning_rate = 5e-5
num_hidden_units = 50

model = ShallowRegressionLSTM(num_sensors=len(features), hidden_units=num_hidden_units)
loss_function = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()

    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")

def test_model(data_loader, model, loss_function):

    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")


print("Untrained test\n--------")
test_model(test_loader, model, loss_function)
print()

for ix_epoch in range(5):
    print(f"Epoch {ix_epoch}\n---------")
    train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_model(test_loader, model, loss_function)
    print()

Untrained test
--------
Test loss: 0.16922074345775934

Epoch 0
---------
Train loss: 0.11679467067696643
Test loss: 0.11720271513819598

Epoch 1
---------
Train loss: 0.09495374297824032
Test loss: 0.1204059234366629

Epoch 2
---------
Train loss: 0.08501729602644914
Test loss: 0.11935944060845513

Epoch 3
---------
Train loss: 0.07821969228286471
Test loss: 0.12091042280183448

Epoch 4
---------
Train loss: 0.07291925423747952
Test loss: 0.1219807500096495



In [ ]:
def predict(data_loader, model):

    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)

    return output


train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

ystar_col = "Model forecast"
df_train[ystar_col] = predict(train_eval_loader, model).numpy()
df_test[ystar_col] = predict(test_loader, model).numpy()

df_out = pd.concat((df_train, df_test))[[target, ystar_col]]

for c in df_out.columns:
    df_out[c] = df_out[c] * target_stdev + target_mean

print(df_out)

<ipython-input-52-220cab55023c>:16: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



       444_lead6  Model forecast
0          0.889        0.935987
1          0.728        0.732237
2          0.780        0.688649
3          0.345        0.488578
4          0.367        0.450853
...          ...             ...
13622      2.147        1.263398
13623      1.864        1.197525
13624      1.422        1.240154
13625      0.873        1.190437
13626      1.084        1.171357

[13627 rows x 2 columns]


<ipython-input-52-220cab55023c>:17: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [ ]:
fig = px.line(df_out, labels=dict(created_at="Date", value="PM2.5 (ug/m3)"))
fig.add_vline(x=test_start, line_width=4, line_dash="dash")
fig.add_annotation(xref="paper", x=0.75, yref="paper", y=0.8, text="Test set start", showarrow=False)
fig.update_layout(
    template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()